In [11]:
import cv2, os
import numpy as np

In [9]:
img_dir = './hub_data/test/images'
img_list = os.listdir(img_dir)
ir_images = [x for x in img_list if x.startswith('IR')]
th_images = [y for y in img_list if y.startswith('TH')]

66012
63679


In [13]:
print(ir_images[0])

IR_A16019_2_20210813_21_001_00014.jpg


In [17]:
ir_pix_value = []
ir_max = 0
ir_min = 256
for img in ir_images[:1000]:
    image = cv2.imread(os.path.join(img_dir,img))
    pix_value = np.mean(image)
    if pix_value > ir_max:
        ir_max = pix_value
    if pix_value < ir_min:
        ir_min = pix_value
    ir_pix_value.append(pix_value)

print(np.mean(ir_pix_value), ir_max, ir_min)

89.63275283600984 116.14512044270833 51.14127278645833


In [18]:
th_pix_value = []
th_max = 0
th_min = 256
for img in th_images[:1000]:
    image = cv2.imread(os.path.join(img_dir,img))
    pix_value = np.mean(image)
    if pix_value > th_max:
        th_max = pix_value
    if pix_value < th_min:
        th_min = pix_value
    th_pix_value.append(pix_value)

print(np.mean(th_pix_value), th_max, th_min)

144.58742820434568 178.95345764160157 102.82081909179688


In [25]:
# class 비율 확인
label_dir = './hub_data/test/labels'
label_list = os.listdir(label_dir)
ir_labels = [x for x in label_list if x.startswith('IR')]
th_labels = [y for y in label_list if y.startswith('TH')]

print(len(ir_labels))
print(len(th_labels))

63679
66012


In [37]:
import pandas as pd
ir_labels[0]

f = open(os.path.join(label_dir, ir_labels[4]), 'r')
x = f.readlines()

'0'

In [23]:
ir_class = [0, 0, 0, 0]
th_class = [0, 0, 0, 0]

In [38]:
for label in ir_labels:
    with open(os.path.join(label_dir, label), 'r') as f:
        lines = f.readlines()
        for line in lines:
            ir_class[int(line[0])] += 1
print(ir_class)

[37139, 16213, 13488, 8092]


In [39]:
for label in th_labels:
    with open(os.path.join(label_dir, label), 'r') as f:
        lines = f.readlines()
        for line in lines:
            th_class[int(line[0])] += 1
print(th_class)

[40781, 16438, 17987, 8739]


In [5]:
# train test split
import os
from glob import glob
import shutil
from sklearn.model_selection import train_test_split

image_files = glob("./hub_data/test/images/IR/*")
file_names = [name.replace(".jpg", "") for name in image_files]
train_names, test_names = train_test_split(file_names, test_size=0.2, random_state=42, shuffle = True)

def move_files_image(file_list, source_path, dest_path):
    for file in file_list:
        image = file.split('\\')[-1] + '.jpg'
        shutil.move(os.path.join(source_path, image), os.path.join(dest_path, image))
    return

def move_files_txt(file_list, source_path, dest_path):
    for file in file_list:
        txt = file.split('\\')[-1] + '.txt'
        shutil.move(os.path.join(source_path, txt), os.path.join(dest_path, txt))
    return

move_files_image(train_names,"./hub_data/test/images/IR/", "./data_IR/train/images/")
move_files_txt(train_names,"./hub_data/test/labels/IR/", "./data_IR/train/labels/")
move_files_image(test_names,"./hub_data/test/images/IR/", "./data_IR/test/images/")
move_files_txt(test_names,"./hub_data/test/labels/IR/", "./data_IR/test/labels/")

# move_files_image(train_names,"./hub_data/test/images/TH/", "./data_TH/train/images/")
# move_files_txt(train_names,"./hub_data/test/labels/TH/", "./data_TH/train/labels/")
# move_files_image(test_names,"./hub_data/test/images/TH/", "./data_TH/test/images/")
# move_files_txt(test_names,"./hub_data/test/labels/TH/", "./data_TH/test/labels/")

In [6]:
import os
from glob import glob
import shutil
from sklearn.model_selection import train_test_split

image_files = glob("./hub_data/test/images/TH/*")
file_names = [name.replace(".jpg", "") for name in image_files]
train_names, test_names = train_test_split(file_names, test_size=0.2, random_state=42, shuffle = True)

def move_files_image(file_list, source_path, dest_path):
    for file in file_list:
        image = file.split('\\')[-1] + '.jpg'
        shutil.move(os.path.join(source_path, image), os.path.join(dest_path, image))
    return

def move_files_txt(file_list, source_path, dest_path):
    for file in file_list:
        txt = file.split('\\')[-1] + '.txt'
        shutil.move(os.path.join(source_path, txt), os.path.join(dest_path, txt))
    return

# move_files_image(train_names,"./hub_data/test/images/IR/", "./data_IR/train/images/")
# move_files_txt(train_names,"./hub_data/test/labels/IR/", "./data_IR/train/labels/")
# move_files_image(test_names,"./hub_data/test/images/IR/", "./data_IR/test/images/")
# move_files_txt(test_names,"./hub_data/test/labels/IR/", "./data_IR/test/labels/")

move_files_image(train_names,"./hub_data/test/images/TH/", "./data_TH/train/images/")
move_files_txt(train_names,"./hub_data/test/labels/TH/", "./data_TH/train/labels/")
move_files_image(test_names,"./hub_data/test/images/TH/", "./data_TH/test/images/")
move_files_txt(test_names,"./hub_data/test/labels/TH/", "./data_TH/test/labels/")

In [12]:
import os, csv
import numpy as np
import pandas as pd
from tqdm import tqdm

txt_file_path = './runs/test/yolo7_val_IR/labels'
save_csv_path = './runs/test/yolo7_val_IR/labels_csv'
os.makedirs(save_csv_path, exist_ok=True)

txt_files = os.listdir(txt_file_path)

for txt_file in tqdm(txt_files):

    with open(os.path.join(txt_file_path, txt_file), 'r') as f:
        lines = f.readlines()

    file_name = txt_file.replace('.txt', '.csv')
    csv_file = open(os.path.join(save_csv_path, file_name), 'w', newline='', encoding='UTF-8')
    csv_writer = csv.writer(csv_file)
    for line in lines:
        line = line.strip().split(' ')
        id, x_c, y_c, w, h, score = list(map(float, line))
        x1, y1, x2, y2 = (x_c - w/2), (y_c- h/2), (x_c + w/h), (y_c + h/2)
        csv_writer.writerow([id, score, x1, y1, x2, y2])
    csv_file.close()

100%|██████████| 11789/11789 [00:29<00:00, 393.70it/s]


In [1]:
[[]]*2

[[], []]